In [5]:
import pandas as pd

In [7]:
def price_storage_contract(
    injection_dates,
    withdrawal_dates,
    prices,
    inj_rate,
    wd_rate,
    max_volume,
    storage_cost,
    inj_cost,
    wd_cost,
    transport_cost
):
    current_volume = 0
    total_cost = 0
    total_revenue = 0
    cash_flow_details = []

    for date, volume in injection_dates:
        if volume > inj_rate:
            raise ValueError(f"Injection on {date} exceeds rate limit")
        if current_volume + volume > max_volume:
            raise ValueError(f"Storage exceeds capacity on {date}")

        price = prices.loc[date]
        cost = price * volume
        inj_fee = inj_cost * (volume / 1e6)
        total_event_cost = cost + inj_fee + transport_cost
        total_cost += total_event_cost
        current_volume += volume

        cash_flow_details.append({
            "date": date,
            "type": "injection",
            "volume": volume,
            "price": price,
            "event_cost": total_event_cost
        })

    for date, volume in withdrawal_dates:
        if volume > wd_rate:
            raise ValueError(f"Withdrawal on {date} exceeds rate limit")
        if current_volume - volume < 0:
            raise ValueError(f"Not enough gas to withdraw on {date}")

        price = prices.loc[date]
        revenue = price * volume
        wd_fee = wd_cost * (volume / 1e6)
        total_event_cost = wd_fee + transport_cost
        total_cost += total_event_cost
        total_revenue += revenue
        current_volume -= volume

        cash_flow_details.append({
            "date": date,
            "type": "withdrawal",
            "volume": volume,
            "price": price,
            "event_revenue": revenue,
            "event_cost": total_event_cost
        })

    if injection_dates and withdrawal_dates:
        start = min(d[0] for d in injection_dates)
        end = max(d[0] for d in withdrawal_dates)
        months = (end.year - start.year) * 12 + (end.month - start.month)
        storage_total_cost = months * storage_cost
        total_cost += storage_total_cost
    else:
        storage_total_cost = 0

    contract_value = total_revenue - total_cost
    return contract_value, cash_flow_details, storage_total_cost

In [9]:
if __name__ == "__main__":
    df = pd.read_csv("Nat_Gas.csv")
    df.head()
    df['Dates'] = pd.to_datetime(df['Dates'], format="%m/%d/%y")

    df.set_index('Dates', inplace=True)

    prices = df['Prices']

    injections = [(pd.Timestamp("2020-10-31"), 1e6)]
    withdrawals = [(pd.Timestamp("2021-02-28"), 1e6)]

    value, details, storage_cost_total = price_storage_contract(
        injection_dates=injections,
        withdrawal_dates=withdrawals,
        prices=prices,
        inj_rate=1.2e6,
        wd_rate=1.2e6,
        max_volume=2e6,
        storage_cost=100000,
        inj_cost=10000,
        wd_cost=10000,
        transport_cost=50000
    )

    print("Contract Value: $", value)
    print("\nBreakdown of cash flows:")
    for event in details:
        print(event)
    print("\nTotal storage cost:", storage_cost_total)

Contract Value: $ 280000.0

Breakdown of cash flows:
{'date': Timestamp('2020-10-31 00:00:00'), 'type': 'injection', 'volume': 1000000.0, 'price': 10.1, 'event_cost': 10160000.0}
{'date': Timestamp('2021-02-28 00:00:00'), 'type': 'withdrawal', 'volume': 1000000.0, 'price': 10.9, 'event_revenue': 10900000.0, 'event_cost': 60000.0}

Total storage cost: 400000
